## 에이전트를 사용하여 구조화된 출력 생성

#### [현재 올바로 동작하지 않는 이슈가 있습니다. 추후 업데이트 예정입니다]

Azure OpenAI Chat Completion 서비스를 기반으로 구축된 에이전트를 사용하여 구조화된 출력을 생성하는 방법을 보여줍니다.

> **중요**
>
>모든 [에이전트 유형](SupportedAgentType.ipynb)이 구조화된 출력을 지원하는 것은 아닙니다. ChatAgent 유형은 호환되는 채팅 클라이언트와 함께 사용할 경우 구조화된 출력을 지원합니다
>
> AzureOpenAIChatClient (Azure OpenAI Chat Completions)    
> OpenAIResponsesClient (OpenAI-compatible Responses API)  
> GitHubOpenAIChatClient (GitHub Models chat endpoint) 


In [ ]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

에이전트로부터 얻고자 하는 출력의 구조를 나타내는 Pydantic 모델을 정의

In [ ]:
from pydantic import BaseModel

class PersonInfo(BaseModel):
    """Information about a person."""
    name: str | None = None
    age: int | None = None
    occupation: str | None = None

이제 Azure OpenAI Chat 클라이언트를 사용하여 에이전트를 생성할 수 있습니다.

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# Create the agent using Azure OpenAI Chat Client
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
   name="HelpfulAssistant",
   instructions="You are a helpful assistant that extracts person information from text."#
)


### ⚠️ Agent Framework의 Structured Outputs 버그
### 이하 부분은 추후 수정될 예정입니다. 랩에서는 수행하지 않습니다.

**문제**: Agent Framework의 `agent.run(response_format=PersonInfo)`가 `response.value`에 structured output을 반환하지 않습니다.

**원인**: Agent Framework가 내부적으로 `beta.chat.completions.parse()` 대신 일반 `chat.completions.create()`를 사용하고 있어서, Azure OpenAI의 structured outputs 기능을 완전히 활용하지 못합니다.

**해결 방법**: 현재는 Azure OpenAI Python SDK를 직접 사용하여 `beta.chat.completions.parse()`를 호출하면 정상 작동합니다. 하지만, 추후 SDK가 업데이트 되면 다시 수행하는 것으로 진행하겠습니다.

Response의 value 속성을 통해 PersonInfo 모델 인스턴스에 직접 액세스할 수 있습니다.

In [ ]:
response = await agent.run(
    "Please provide information about John Smith, who is a 35-year-old software engineer.",
    response_format=PersonInfo
)
print("Response : " + str(response))

if response.value:
    person_info = response.value
    print(f"Name: {person_info.name}, Age: {person_info.age}, Occupation: {person_info.occupation}")
else:
    print("No structured data found in response")

스트리밍 시 에이전트 응답은 일련의 업데이트로 스트리밍됩니다. 구조화된 출력을 얻으려면 모든 업데이트를 수집한 다음 최종 응답 값에 액세스해야 합니다.

In [ ]:
from agent_framework import AgentResponse

query = "Tell me about Jane Doe, a 28-year-old graphic designer."

# Get structured response from streaming agent using AgentResponse.from_agent_response_generator
# This method collects all streaming updates and combines them into a single AgentResponse
final_response = await AgentResponse.from_agent_response_generator(
    agent.run_stream(query, response_format=PersonInfo),
    output_format_type=PersonInfo,
)

if final_response.value:
    person_info = final_response.value
    print(f"Name: {person_info.name}, Age: {person_info.age}, Occupation: {person_info.occupation}")